# Memory Management in NumPy

<ins>*some notes are alson written on Notion app*</ins>

## Internal memory layout of an ndarray

In [1]:
import numpy as np

In [2]:
x = np.arange(5)

In [3]:
x.flags

  C_CONTIGUOUS : True
  F_CONTIGUOUS : True
  OWNDATA : True
  WRITEABLE : True
  ALIGNED : True
  WRITEBACKIFCOPY : False
  UPDATEIFCOPY : False

In [58]:
c_array = np.random.rand(10000, 10000)

In [59]:
c_array.strides

(80000, 8)

In [60]:
f_array = np.asfortranarray(c_array)

In [61]:
f_array.strides

(8, 80000)

In [62]:
c_array.flags

  C_CONTIGUOUS : True
  F_CONTIGUOUS : False
  OWNDATA : True
  WRITEABLE : True
  ALIGNED : True
  WRITEBACKIFCOPY : False
  UPDATEIFCOPY : False

In [63]:
f_array.flags

  C_CONTIGUOUS : False
  F_CONTIGUOUS : True
  OWNDATA : True
  WRITEABLE : True
  ALIGNED : True
  WRITEBACKIFCOPY : False
  UPDATEIFCOPY : False

In [64]:
np.may_share_memory(c_array, f_array)

False

now let's check some operations on these two arrays to see the difference

In [73]:
def sum_row(arr) :
    # return np.sum(arr[0, :])
    return np.sum(arr, axis=1)

In [74]:
def sum_col(arr) :
    return np.sum(arr, axis=0)

In [75]:
%timeit sum_row(c_array)

32.3 ms ± 916 µs per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [76]:
%timeit sum_row(f_array)

43 ms ± 74.2 µs per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [77]:
%timeit sum_col(c_array)

44.5 ms ± 272 µs per loop (mean ± std. dev. of 7 runs, 10 loops each)


In [78]:
%timeit sum_col(f_array)

33.9 ms ± 133 µs per loop (mean ± std. dev. of 7 runs, 10 loops each)


## Views ans Copies

In [83]:
x = np.random.randint(1, 10, size=(3, 3))
x

array([[4, 3, 3],
       [7, 5, 5],
       [9, 4, 9]])

In [84]:
y = x
np.may_share_memory(x, y)

True

In [85]:
y[0, :] = -1
y

array([[-1, -1, -1],
       [ 7,  5,  5],
       [ 9,  4,  9]])

In [86]:
x

array([[-1, -1, -1],
       [ 7,  5,  5],
       [ 9,  4,  9]])

so as we can see, when they share memory, if you change one of them, the other one will be changed too<br>
in this case, we say **y is a view of x**

___

In [88]:
x = np.random.randint(1, 10, size=(3, 3))
x

array([[1, 8, 6],
       [2, 7, 6],
       [3, 2, 9]])

In [97]:
y = x[:, 0] 
y 
# here y is only the first column of x

array([1, 2, 3])

In [98]:
np.may_share_memory(x, y)

True

In [99]:
y[:] = 0
y

array([0, 0, 0])

In [100]:
x

array([[0, 8, 6],
       [0, 7, 6],
       [0, 2, 9]])

in this example, although y is a just a part of x, still it's a view and if you change y, x will be changed too

___

In [121]:
x = np.arange(1, 10).reshape(3, 3)
x

array([[1, 2, 3],
       [4, 5, 6],
       [7, 8, 9]])

In [122]:
y = np.empty((3, 3))
y

array([[0.00000000e+000, 1.00937611e-320, 0.00000000e+000],
       [0.00000000e+000, 0.00000000e+000, 1.16095484e-028],
       [1.46899930e+179, 9.08367237e+223, 1.16466228e-028]])

In [123]:
y[0, :] = x[0, :]
y # first row of y is now has the same element as first row of x

array([[1.00000000e+000, 2.00000000e+000, 3.00000000e+000],
       [0.00000000e+000, 0.00000000e+000, 1.16095484e-028],
       [1.46899930e+179, 9.08367237e+223, 1.16466228e-028]])

In [124]:
y[0, :] = 0
y

array([[0.00000000e+000, 0.00000000e+000, 0.00000000e+000],
       [0.00000000e+000, 0.00000000e+000, 1.16095484e-028],
       [1.46899930e+179, 9.08367237e+223, 1.16466228e-028]])

In [125]:
x

array([[1, 2, 3],
       [4, 5, 6],
       [7, 8, 9]])

after changing y, x is the same as it was before,
so they should not share memory in this case :

In [126]:
np.may_share_memory(x, y)

False

___